In [1]:
import lark

In [73]:
from lark import Lark, Transformer, Token, Tree


In [ ]:
# document: entry_conditions_section? \
#           prompt_section? \
#           continue_conditions_section? \
#           locals_section? \
#           entry_conditions_section? \
#           set_state_section? \
#           subnodes_section \
#           set_state_after_section?


In [85]:
from dataclasses import dataclass

In [93]:
@dataclass
class Predicate:
    verb : str
    variable : str

In [104]:
@dataclass
class Variable:
    namespace : str
    variable : str

In [112]:
@dataclass
class DerivedCondition:
    operator : str
    elem1 : "Any"
    elem2 : "Any"

In [132]:
@dataclass
class NeuralGeneration:
    prefix : str

In [133]:
@dataclass
class Inflect:
    inflect_token : "NLG"
    inflect_entity : Variable

In [135]:
@dataclass
class Prompt:
    prompt_name : str
    condition : "Any"
    nlg : "NLG"

In [138]:
@dataclass
class String:
    string : str

In [197]:
@dataclass
class Assignment:
    left : "Any"
    right : "Any"

In [226]:
@dataclass
class NLG:
    elems : "List"

In [232]:
@dataclass
class OneOf:
    choices : "List"

In [260]:
@dataclass
class Constant:
    constant : str

In [274]:
@dataclass
class NLGHelper:
    func_name : str
    args : "List[str]"

In [281]:
class SupernodeMaker(Transformer):
    def variable(self, tok):
        return Variable(str(tok[0]), str(tok[1]))
    def condition__variable(self, tok):
        return self.variable(tok)
    def condition__predicate(self, tok):
        return Predicate(verb=str(tok[0]), variable=tok[1])
    def nlg__variable(self, tok):
        return self.variable(tok)

    def condition(self, tok):
        if isinstance(tok, list):
            if len(tok) == 1: return tok[0]
            if isinstance(tok[0], Token) and tok[0].type == 'condition__OP':
                return tok[1]
            # condition "and" condition
            if isinstance(tok[1], Token) and tok[1] == 'and':
                return DerivedCondition('and', tok[0], tok[2])
            if isinstance(tok[1], Token) and tok[1] == 'or':
                return DerivedCondition('or', tok[0], tok[2])
        return tok
    
    def entry_conditions_section(self, tok):
        if len(tok):
            return tok[0]
        return None
    
    def nlg__ESCAPED_STRING(self, tok):
        return String(str(tok.value)[1:-1])

    def nlg__PUNCTUATION(self, tok):
        return String(str(tok.value))

    def nlg__val(self, tok):
        return tok[2]

    def nlg__neural_generation(self, tok):
        return NeuralGeneration(tok[0])
    
    def nlg__val(self, tok):
        return tok[0]
    
    def nlg__one_of(self, tok):
        return OneOf(tok[1])
    
    def nlg__constant(self, tok):
        return Constant(tok[0].value)
    
    def nlg__inflect(self, tok):
        return tok[0]
    
    def nlg__STRING(self, tok):
        return tok
    
    def nlg__helper(self, tok):
        func_name = tok[0].value
        args = tok[1:]
        return NLGHelper(func_name, args)
    
    def nlg(self, tok):
        if isinstance(tok, list):
            if len(tok) == 1: return tok[0]
            return tok
        return NLG(tok)
    
    def prompt(self, tok):
        if len(tok) == 3:
            prompt_name, condition, nlg = tok
        else:
            prompt_name, nlg = tok
            condition = None
        return Prompt(prompt_name.value, condition, nlg)
    
    def prompt_section(self, tok):
        return {"Prompts": tok}

    def continue_conditions_section(self, tok):
        return tok
        
    def assignment(self, tok):
        return Assignment(tok[0], tok[1])
        
    def locals_section(self, tok):
        return {"Locals": tok}
    def set_state_section(self, tok):
        return {"Set_State": tok}
    def subnodes_section(self, tok):
        return {"Subnodes Section": tok}
    def set_state_after_section(self, tok):
        return {"Set State After": tok}

    def document(self, tok):
        for x in tok:
            print(x)
        return tok

In [282]:
with open('grammar.lark', 'r') as f:
    grammar = f.read()
#print(grammar)
json_parser = Lark(grammar, start='document', parser='lalr', transformer=SupernodeMaker())
with open('test1.camel', 'r') as f:
    text = f.read()
tree = json_parser.parse(text)
#print(json.dumps(tree, indent=2, default=str))
print(tree)

DerivedCondition(operator='and', elem1=Predicate(verb='IS_NONE', variable=Variable(namespace='State', variable='User__FavoriteFood')), elem2=Predicate(verb='IS_NONE', variable=Variable(namespace='State', variable='User__FavoriteFood')))
{'Prompts': [Prompt(prompt_name='favorite_food', condition=None, nlg=String(string="What's your favorite food?"))]}
[Predicate(verb='IS_TRUE', variable=Variable(namespace='Flags', variable='FOOD__user_mentioned_food'))]
{'Locals': [Assignment(left=Variable(namespace='Locals', variable='intro_statement'), right=[OneOf(choices=String(string='Oh yeah')), Variable(namespace='Utilities', variable='cur_entity_talkable_lower'), String(string='is'), OneOf(choices=String(string="such an amazing choice. It's one of my favorite foods up here in the cloud."))])]}
{'Set_State': [Assignment(left=Variable(namespace='State', variable='User__FavoriteFood'), right=Variable(namespace='Utilities', variable='cur_entity')), Assignment(left=Variable(namespace='State', variabl

In [213]:
tree

[DerivedCondition(operator='and', elem1=Predicate(verb='IS_NONE', variable=Variable(namespace='State', variable='User__FavoriteFood')), elem2=Predicate(verb='IS_NONE', variable=Variable(namespace='State', variable='User__FavoriteFood'))),
 {'Prompts': [Prompt(prompt_name='Prompt favorite_food', condition=None, nlg=String(string="What's your favorite food?"))]},
 [Predicate(verb='IS_TRUE', variable=Variable(namespace='Flags', variable='FOOD__user_mentioned_food'))],
 'locals_section',
 'set_state_section',
 'subnodes_section',
 'set_state_after_section']

In [10]:
tree

Tree(Token('RULE', 'document'), [Tree(Token('RULE', 'entry_conditions_section'), [Tree(Token('RULE', 'condition'), [Tree('condition__predicate', [Token('condition__PREDICATE', 'IS_NONE'), Token('condition__OP', '('), Tree('condition__variable', [Token('condition__utils__NAMESPACE', 'State'), Token('condition__PARAM_ARG_VALUE', 'User__FavoriteFood')]), Token('condition__CP', ')')])])]), Tree(Token('RULE', 'prompt_section'), [Tree(Token('RULE', 'prompt'), [Token('PROMPT_NAME', 'Prompt favorite_food'), Token('OB', '['), Token('CB', ']'), Tree(Token('RULE', 'nlg'), [Tree(Token('RULE', 'nlg'), [Token('nlg__ESCAPED_STRING', '"What\'s your favorite food?"')])])])]), Tree(Token('RULE', 'continue_conditions_section'), [Tree(Token('RULE', 'condition'), [Tree('condition__predicate', [Token('condition__PREDICATE', 'IS_TRUE'), Token('condition__OP', '('), Tree('condition__variable', [Token('condition__utils__NAMESPACE', 'Flags'), Token('condition__PARAM_ARG_VALUE', 'FOOD__user_mentioned_food')]), T

In [204]:

print(json_parser.parse(text).pretty())

locals [Assignment(left=Variable(namespace='Locals', variable='intro_statement'), right=[Tree('nlg__one_of', [Token('nlg__OP', '('), String(string='Ah yes'), Token('nlg__COMMA', ','), String(string='Oh yeah'), Token('nlg__COMMA', ','), Token('nlg__CP', ')')]), Token('nlg__CP', ')'), Tree('nlg__inflect', [Token('nlg__OP', '('), Token('nlg__EQ', '='), String(string='is'), Token('nlg__COMMA', ','), Token('nlg__EQ', '='), Variable(namespace='Utilities', variable='cur_entity'), Token('nlg__CP', ')')]), Tree('nlg__one_of', [Token('nlg__OP', '('), String(string='one of my favorite things to eat up here in the cloud.'), Token('nlg__COMMA', ','), String(string="such an amazing choice. It's one of my favorite foods up here in the cloud."), Token('nlg__CP', ')')])])]
DerivedCondition(operator='and', elem1=Predicate(verb='IS_NONE', variable=Variable(namespace='State', variable='User__FavoriteFood')), elem2=Predicate(verb='IS_NONE', variable=Variable(namespace='State', variable='User__FavoriteFood'

AttributeError: 'list' object has no attribute 'pretty'

In [89]:
text = """
Subnode::SubnodeName { 
    {Hi}
    ,
    {Do you like}
    val ( UserFavoriteFood )
    ?
}
"""
print(json_parser.parse(text).pretty())

document
  subnodes
    subnode
      Subnode::SubnodeName
      nlg_generation
        nlg_generation
          nlg_generation	Hi
        nlg_generation	,
        nlg_generation
          nlg_generation	Do you like
        nlg_generation
          val	UserFavoriteFood 
        nlg_generation	?



In [ ]:
              prompts? 
              entry_state_conditions_takeover? 
              continue_conditions?
              locals?
              set_state?
              subnodes
set_state_after?

value
  nlg_generation_wrap
    nlg_generation
      nlg_generation	Hi
    nlg_generation	,
    
    nlg_generation
      nlg_generation	Do you like
    nlg_generation
      val	UserFavoriteFood 
    nlg_generation	?

